In [1]:
import pandas as pd

In [2]:
merged_data = pd.read_csv('/content/drive/MyDrive/fraud detector/merged_data.csv')
creditcard_data = pd.read_csv('/content/drive/MyDrive/fraud detector/creditcard.csv')

In [3]:
merged_data.head()

,Unnamed: 0,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country
0,634,247547,2015-06-28 03:00:34,2015-08-09 03:57:29,47.0,KIXYSVCHIPQBR,SEO,Safari,F,30,16778864,0,Australia
1,635,220737,2015-01-28 14:21:11,2015-02-11 20:28:28,15.0,PKYOWQKWGJNJI,SEO,Chrome,F,34,16842045,0,Thailand
2,636,390400,2015-03-19 20:49:09,2015-04-11 23:41:23,44.0,LVCSXLISZHVUO,Ads,IE,M,29,16843656,0,China
3,637,69592,2015-02-24 06:11:57,2015-05-23 16:40:14,55.0,UHAUHNXXUADJE,Direct,Chrome,F,30,16938732,0,China
4,638,174987,2015-07-07 12:58:11,2015-11-03 04:04:30,51.0,XPGPMOHIDRMGE,SEO,Chrome,F,37,16971984,0,Thailand


In [6]:
creditcard_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
# Convert timestamps to datetime format
merged_data['signup_time'] = pd.to_datetime(merged_data['signup_time'], errors='coerce')
merged_data['purchase_time'] = pd.to_datetime(merged_data['purchase_time'], errors='coerce')

# Transaction frequency
merged_data['transaction_frequency'] = merged_data.groupby('user_id')['purchase_time'].transform('count')

# Transaction velocity
merged_data['transaction_velocity'] = (merged_data['purchase_time'] - merged_data['signup_time']).dt.total_seconds() / merged_data['transaction_frequency']

# Time-based features
merged_data['hour_of_day'] = merged_data['purchase_time'].dt.hour
merged_data['day_of_week'] = merged_data['purchase_time'].dt.dayofweek

In [5]:
merged_data.head()

,Unnamed: 0,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,transaction_frequency,transaction_velocity,hour_of_day,day_of_week
0,634,247547,2015-06-28 03:00:34,2015-08-09 03:57:29,47.0,KIXYSVCHIPQBR,SEO,Safari,F,30,16778864,0,Australia,1,3632215.0,3,6
1,635,220737,2015-01-28 14:21:11,2015-02-11 20:28:28,15.0,PKYOWQKWGJNJI,SEO,Chrome,F,34,16842045,0,Thailand,1,1231637.0,20,2
2,636,390400,2015-03-19 20:49:09,2015-04-11 23:41:23,44.0,LVCSXLISZHVUO,Ads,IE,M,29,16843656,0,China,1,1997534.0,23,5
3,637,69592,2015-02-24 06:11:57,2015-05-23 16:40:14,55.0,UHAUHNXXUADJE,Direct,Chrome,F,30,16938732,0,China,1,7640897.0,16,5
4,638,174987,2015-07-07 12:58:11,2015-11-03 04:04:30,51.0,XPGPMOHIDRMGE,SEO,Chrome,F,37,16971984,0,Thailand,1,10249579.0,4,1


In [11]:
merged_data = merged_data.drop(columns=['Unnamed: 0'], errors='ignore')

In [12]:
# Perform one-hot encoding
merged_data = pd.get_dummies(merged_data, columns=['source', 'browser', 'sex', 'country'], drop_first=True)

In [13]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le = LabelEncoder()

# Apply label encoding to device_id
if 'device_id' in merged_data.columns:
    merged_data['device_id'] = le.fit_transform(merged_data['device_id'])

In [17]:
# Fraud_Data.csv
X_fraud = merged_data.drop(columns=['class', 'user_id', 'signup_time', 'purchase_time', 'ip_address'])
y_fraud = merged_data['class']

# creditcard.csv
X_credit = creditcard_data.drop(columns=['Class', 'Time'])  # Exclude 'Time' as it's anonymized
y_credit = creditcard_data['Class']

In [18]:
from sklearn.model_selection import train_test_split

# Fraud Data
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(
    X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud
)

# Credit Card Data
X_train_credit, X_test_credit, y_train_credit, y_test_credit = train_test_split(
    X_credit, y_credit, test_size=0.2, random_state=42, stratify=y_credit
)

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "MLP": MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
}

# Train and evaluate models for Fraud_Data.csv
results_fraud = {}
for name, model in models.items():
    model.fit(X_train_fraud, y_train_fraud)
    y_pred = model.predict(X_test_fraud)
    y_prob = model.predict_proba(X_test_fraud)[:, 1]

    # Evaluate performance
    report = classification_report(y_test_fraud, y_pred, output_dict=True)
    auc = roc_auc_score(y_test_fraud, y_prob)
    results_fraud[name] = {
        "Precision": report['1']['precision'],
        "Recall": report['1']['recall'],
        "F1-Score": report['1']['f1-score'],
        "AUC-ROC": auc
    }

# Train and evaluate models for creditcard.csv
results_credit = {}
for name, model in models.items():
    model.fit(X_train_credit, y_train_credit)
    y_pred = model.predict(X_test_credit)
    y_prob = model.predict_proba(X_test_credit)[:, 1]

    # Evaluate performance
    report = classification_report(y_test_credit, y_pred, output_dict=True)
    auc = roc_auc_score(y_test_credit, y_prob)
    results_credit[name] = {
        "Precision": report['1']['precision'],
        "Recall": report['1']['recall'],
        "F1-Score": report['1']['f1-score'],
        "AUC-ROC": auc
    }

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/xgboost/c

In [21]:
# Results for Fraud_Data.csv
results_fraud_df = pd.DataFrame(results_fraud).T
print("Fraud Data Model Performance:")
print(results_fraud_df)

Fraud Data Model Performance:
                     Precision    Recall  F1-Score   AUC-ROC
Logistic Regression   0.000000  0.000000  0.000000  0.756042
Decision Tree         0.517406  0.575387  0.544858  0.759524
Random Forest         1.000000  0.547677  0.707741  0.772349
Gradient Boosting     0.997033  0.547677  0.706996  0.780963
XGBoost               0.938547  0.547677  0.691714  0.771529
MLP                   0.999257  0.547677  0.707555  0.773815


In [22]:
# Results for creditcard.csv
results_credit_df = pd.DataFrame(results_credit).T
print("\nCredit Card Data Model Performance:")
print(results_credit_df)


Credit Card Data Model Performance:
                     Precision    Recall  F1-Score   AUC-ROC
Logistic Regression   0.828947  0.642857  0.724138  0.958163
Decision Tree         0.755102  0.755102  0.755102  0.877340
Random Forest         0.941860  0.826531  0.880435  0.952811
Gradient Boosting     0.529412  0.183673  0.272727  0.346886
XGBoost               0.890110  0.826531  0.857143  0.971138
MLP                   0.879121  0.816327  0.846561  0.972474


In [ ]:
import mlflow
mlflow.set_tracking_uri("file:///content/mlruns")

In [25]:
import mlflow
import mlflow.sklearn

# Start an MLflow run
mlflow.set_experiment("Fraud Detection Models")

# Example: Logging Random Forest for Fraud_Data.csv
with mlflow.start_run(run_name="RandomForest_Fraud"):
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train_fraud, y_train_fraud)

    # Predictions
    y_pred = rf.predict(X_test_fraud)
    y_prob = rf.predict_proba(X_test_fraud)[:, 1]

    # Metrics
    report = classification_report(y_test_fraud, y_pred, output_dict=True)
    auc = roc_auc_score(y_test_fraud, y_prob)

    # Log parameters and metrics
    mlflow.log_param("model", "Random Forest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_metric("precision", report['1']['precision'])
    mlflow.log_metric("recall", report['1']['recall'])
    mlflow.log_metric("f1_score", report['1']['f1-score'])
    mlflow.log_metric("auc_roc", auc)

    # Log the model
    mlflow.sklearn.log_model(rf, "random_forest_model")

2025/02/10 21:10:17 INFO mlflow.tracking.fluent: Experiment with name 'Fraud Detection Models' does not exist. Creating a new experiment.
2025/02/10 21:11:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [24]:
# !pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.1 MB/s eta 0:00:00


In [27]:
from google.colab import files
!zip -r /content/mlruns.zip /content/mlruns
files.download("/content/mlruns.zip")

  adding: content/mlruns/ (stored 0%)
  adding: content/mlruns/0/ (stored 0%)
  adding: content/mlruns/0/meta.yaml (deflated 25%)
  adding: content/mlruns/.trash/ (stored 0%)
  adding: content/mlruns/285413150680008004/ (stored 0%)
  adding: content/mlruns/285413150680008004/meta.yaml (deflated 30%)
  adding: content/mlruns/285413150680008004/46117f1342894c939c923a531537cd55/ (stored 0%)
  adding: content/mlruns/285413150680008004/46117f1342894c939c923a531537cd55/params/ (stored 0%)
  adding: content/mlruns/285413150680008004/46117f1342894c939c923a531537cd55/params/n_estimators (stored 0%)
  adding: content/mlruns/285413150680008004/46117f1342894c939c923a531537cd55/params/model (stored 0%)
  adding: content/mlruns/285413150680008004/46117f1342894c939c923a531537cd55/meta.yaml (deflated 45%)
  adding: content/mlruns/285413150680008004/46117f1342894c939c923a531537cd55/artifacts/ (stored 0%)
  adding: content/mlruns/285413150680008004/46117f1342894c939c923a531537cd55/artifacts/random_fores

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>